In [ ]:
# Imports
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow_datasets as tfds


In [ ]:
# Version check
import google.protobuf

print("TF:", tf.__version__)
print("TFDS:", tfds.__version__)
print("protobuf:", google.protobuf.__version__)


In [ ]:
# Dataset load
# Reload dataset with a proper 80/20 split
(train_ds, val_ds), info = tfds.load(
    "tf_flowers",
    split=["train[:80%]", "train[80%:]"],  # 80% train, 20% validation
    with_info=True,
    as_supervised=True
)


In [ ]:
# Preprocessing
IMG_SIZE = 224   # standard input size for many pretrained models
BATCH_SIZE = 32  # how many images to feed the model at once

# Function to resize and normalize each image
def format_example(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))  # resize to 224x224
    image = image / 255.0  # normalize pixel values [0,255] -> [0,1]
    return image, label

# Apply preprocessing to the dataset and shuffle and batch the data
train_ds = train_ds.map(format_example).shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

val_ds = val_ds.map(format_example).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


In [ ]:
# Visualization of sample images
# Take one batch from the dataset
for images, labels in train_ds.take(1):
    plt.figure(figsize=(10, 10))
    
    for i in range(9):  # show 9 images
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy())  # convert Tensor to NumPy for plotting
        plt.title(int(labels[i].numpy()))  # show class index as title
        plt.axis("off")


In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import Sequential, optimizers

# Number of flower classes (from dataset info)
num_classes = info.features["label"].num_classes
print("Number of classes:", num_classes)

# Load EfficientNetB0 base model (pretrained on ImageNet)
base_model = EfficientNetB0(
    weights="imagenet",        # use pretrained weights
    include_top=False,         # exclude the original ImageNet classifier
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)

# Freeze the base model (so its weights don't change initially)
base_model.trainable = False

# Build our custom model on top
model = Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),        # reduce features
    layers.Dropout(0.5),                    # prevent overfitting
    layers.Dense(num_classes, activation="softmax")  # output layer
])

# Compile the model
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# Summary of the architecture
model.summary()


In [22]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

callbacks = [
    EarlyStopping(
        monitor="val_accuracy",
        patience=3,
        restore_best_weights=True
    ),
    ModelCheckpoint(
        filepath="best_model.h5",
        monitor="val_accuracy",
        save_best_only=True,
        save_weights_only=True,
        verbose=1
    )
]


In [23]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=callbacks
)


Epoch 1/10
92/92 [==============================] - ETA: 0s - loss: 1.6301 - accuracy: 0.2323
Epoch 1: val_accuracy improved from -inf to 0.21662, saving model to best_model.h5
92/92 [==============================] - 90s 980ms/step - loss: 1.6301 - accuracy: 0.2323 - val_loss: 1.6086 - val_accuracy: 0.2166
Epoch 2/10
92/92 [==============================] - ETA: 0s - loss: 1.6304 - accuracy: 0.2238
Epoch 2: val_accuracy did not improve from 0.21662
92/92 [==============================] - 90s 983ms/step - loss: 1.6304 - accuracy: 0.2238 - val_loss: 1.6068 - val_accuracy: 0.2166
Epoch 3/10
92/92 [==============================] - ETA: 0s - loss: 1.6325 - accuracy: 0.2183
Epoch 3: val_accuracy did not improve from 0.21662
92/92 [==============================] - 91s 991ms/step - loss: 1.6325 - accuracy: 0.2183 - val_loss: 1.6042 - val_accuracy: 0.2166
Epoch 4/10
92/92 [==============================] - ETA: 0s - loss: 1.6288 - accuracy: 0.2265
Epoch 4: val_accuracy did not improve from 